## FINAL PROJECT - 02456 DEEP LEARNING
### SEGMENTATION OF CAR PARTS 
### COLLABORATION WITH DELOITTE CONSULTING

#### Authors
>*Sanem Leblebici - s222448*

>*Michal Lehwark - s222999*

>*Ari Menachem - s163956*

>*Elli Georgiou - s223408*


In [ ]:
from tqdm import tqdm
import torch
import numpy as np
import torch.nn as nn
from modules.model import UNET, UNET_RESNET
from modules.unetPP import UNetPP 
from modules.settings import DATA_PATH, NUM_EPOCHS,BATCH_SIZE,SPLIT_RATIO, LEARNING_RATE, device
from modules.utils import get_data_loaders, pixel_accuracy, save_metric_to_file, mean_pixel_accuracy, save_dice_loss_to_file
from modules.resizeImages import resize_with_pad
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from PIL import Image
import os
import cv2
from typing import Tuple
from torchsummary import summary
from torchvision import transforms as T
from sklearn.metrics import confusion_matrix


### Load the trained model

In [ ]:
mdl = UNET()
# mdl = UNET_RESNET(3)
# mdl = UNetPP()

## Can uncomment the next line if your device has gpu
# mdl.load_state_dict(torch.load('./models/unet_nov18.pth'))

model_name = "unet_dec2_3.pth"
model_type = str(type(mdl))

mdl.load_state_dict(torch.load('./models/old/dec2/'+model_name, map_location=torch.device('cpu')))
# mdl.load_state_dict(torch.load('./models/experiment_outputs/output_models_arrays_1_unet/'+model_name, map_location=torch.device('cpu')))
print(f"{model_type} loaded: {model_name}")

### TEST LOOP

In [ ]:
masks_path = './data/carseg_data/test_arrays_mask_corrected'
# masks_path = "./data/carseg_data/FINAL_select_augmented_arrays"
# masks_path = "./data/carseg_data/unused/arrays_no_bg_augment"
torch.set_printoptions(sci_mode=False)

# Create a list to store the model outputs
model_outputs = []
model_outputs_images = []
model_outputs_gt = []
model_output_pixelAcc = [] 
filenames_in_order = []

# Set the model to evaluation mode
mdl.eval()

# Define the transformation to be applied to the input images
transform = transforms.Compose([transforms.ToTensor()])
reverse_transform = transforms.Compose([transforms.ToPILImage()])
how_many_images_to_test = 30
index = 0

for filename in os.listdir(masks_path):
  mask_path = os.path.join(masks_path, filename)
  mask = np.load(mask_path, allow_pickle=True)
  
        
  img = mask[:, :, :3]
  mask_split = mask[:, :, 3]
  mask_split = mask_split.astype(int)
  one_hot_encoded = np.eye(10, dtype=int)[mask_split.squeeze()]
  dice_mask = np.expand_dims(one_hot_encoded.transpose(2,0,1), axis=0)
        
  img = transform(img)
  img_rev = reverse_transform(img)
            
  mdl.eval()
 # Make the prediction
  with torch.no_grad():
    img = img.unsqueeze(0)
    filenames_in_order.append(filename)
    output = mdl(img)
    # sigmoid_output = torch.sigmoid(output)
    argmax_output = torch.argmax(output, dim=1)

    # Calculate pixel accuracy
    accuracy = pixel_accuracy(argmax_output[0], torch.tensor(mask_split))
    model_output_pixelAcc.append(accuracy)
    
  # Store the output in the list
  model_outputs.append(argmax_output[0])
  model_outputs_images.append(img_rev)
  model_outputs_gt.append(torch.tensor(mask_split))

  #detect for first 10 images
  if index == how_many_images_to_test:
    break
  index = index + 1

# The `model_outputs` list now contains the model's output for each test image
# Calculate and print mean pixel accuracy
mean_accuracy = mean_pixel_accuracy(model_outputs, model_outputs_gt)


# Save scores to a file
mean_dice = save_dice_loss_to_file("./dice_scores.txt", model_outputs, model_outputs_gt, filenames_in_order)
save_metric_to_file("./scores.txt", model_type, model_name, mean_accuracy, mean_dice)

### COLORFUL SEGMENTATION PLOT AND LOSS PLOTS

In [ ]:
def plot_loss_from_txt(file_path):
    # Read data from the text file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Extract data from each line
    indices, train_losses, val_losses = [], [], []
    for line in lines[1:]:  # Assuming the first line contains column headers
        index, train_loss, val_loss = map(float, line.strip().split())
        indices.append(index)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

    # Plot the training and validation losses
    plt.figure(figsize=(10, 5))

    # Plot Training Loss
    plt.subplot(1, 2, 1)
    plt.plot(indices, train_losses, label='Train Loss', marker='o', linestyle='-')
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Validation Loss
    plt.subplot(1, 2, 2)
    plt.plot(indices, val_losses, label='Validation Loss', marker='o', linestyle='-')
    plt.title('Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Show the plots
    plt.tight_layout()
    plt.show()

In [ ]:
plot_loss_from_txt('./models/loss_data_nov26_1.txt')

In [ ]:
# Define color mapping for each class
color_mapping = {
        0: (0,0,0),
        1: (250, 149, 10),
        2: (19, 98, 19),
        3: (249, 249, 10),
        4: (10, 248, 250),
        5: (149, 7, 149),
        6: (5, 249, 9),
        7: (20, 19, 249),
        8: (249, 9, 250),
        9: (150, 150, 150),
    }

def colorize_segmentation_mask(segmentation_tensor_or_mask, mask = False):
    # Convert the PyTorch tensor to a NumPy array
    if mask == False:
        segmentation_array = segmentation_tensor_or_mask.squeeze(0).cpu().numpy()
        
    else:
        segmentation_array = segmentation_tensor_or_mask

    # Create a mapping from actual class values to color values
    class_to_color = {class_value: color_mapping[class_value] for class_value in np.unique(segmentation_array)}

    # Create an RGB image with the colored segmentation mask
    colored_mask = np.zeros((256, 256, 3), dtype=np.uint8)
    for class_value, color in class_to_color.items():
        colored_mask[segmentation_array == class_value] = color

    return colored_mask

for image_index_to_display in range(10):
    print(filenames_in_order[image_index_to_display])
    segmentation_tensor = model_outputs[image_index_to_display]
    mask_tensor = model_outputs_gt[image_index_to_display]

    pixel_acc_val = model_output_pixelAcc[image_index_to_display]
    image_tensor = model_outputs_images[image_index_to_display]

    plt.subplot(1, 3, 1)
    plt.imshow(colorize_segmentation_mask(mask_tensor, True))
    plt.title('Ground Truth Mask')
    
    plt.subplot(1, 3, 2)
    plt.imshow(image_tensor)
    plt.title('Real Image')

    plt.subplot(1, 3, 3)
    plt.imshow(colorize_segmentation_mask(segmentation_tensor))
    plt.title(f"Pred Mask, pxAcc: {pixel_acc_val:.5f}")

    plt.show()

### TESTING THE PREPROCESSING OF THE MASK

In [ ]:
color_mapping = {
        0: (0,0,0),
        1: (250, 149, 10),
        2: (19, 98, 19),
        3: (249, 249, 10),
        4: (10, 248, 250),
        5: (149, 7, 149),
        6: (5, 249, 9),
        7: (20, 19, 249),
        8: (249, 9, 250),
        9: (150, 150, 150),
    }


#### WHAT WE DO IN DATASET ######
img_path = './data/carseg_data/images/photo/no_segmentation/0035.jpg'
mask_path = './data/carseg_data/arrays/photo_0035.npy'


mask = np.load(mask_path).astype(np.double)
im = mask[:, :, :3]
mask_split = mask[:, :, 3]
mask_split = mask_split//10
mask_split = mask_split.astype(int)
        
        
transform = T.Compose([T.ToTensor()])
img = transform(im)
        
one_hot_encoded = np.eye(10, dtype=int)[mask_split.squeeze()]
ms = torch.from_numpy(one_hot_encoded)
        
img = img.to(torch.float32)
reshaped_mask_tensor = ms.permute(2, 0, 1)
reshaped_mask_tensor = reshaped_mask_tensor.to(torch.float32)


#### NOW YOU CAN COMPRESS IT BACK IN THIS WAY SHOWING 10 UNIQUE CLASSES: SO IT WORKS CORRECTLY####
print("output shape ", reshaped_mask_tensor.shape)
print("output unique ", torch.unique(reshaped_mask_tensor))
argmax_output = torch.argmax(reshaped_mask_tensor, dim=0).long()
print("argmax shape ", argmax_output.shape)
print("argmax unique ", torch.unique(argmax_output))
#### NOW YOU CAN COMPRESS IT BACK IN THIS WAY SHOWING 10 UNIQUE CLASSES: SO IT WORKS CORRECTLY####



##### PLOT THE MASK THAT IS FED INTO THE MODEL ####
# Plotting
num_classes = 10
num_rows = (num_classes + 4) // 5  # Ensure at least 1 row, with max 5 columns
fig, axes = plt.subplots(3, 5, figsize=(20, num_rows * 4))

# Plot original mask
axes[0, 0].imshow(mask, cmap='viridis', vmin=0, vmax=9)
axes[0, 0].set_title('Original Mask')

im = im.astype(np.uint8)
axes[2, 0].imshow(im)
axes[2, 0].set_title('image')

# Plot one-hot encoded tensors for each class
for i in range(num_classes):
    row = (i ) // 5
    col = (i ) % 5
    axes[row, col].imshow(reshaped_mask_tensor[i , :, :], cmap='viridis', vmin=0, vmax=1)
    axes[row, col].set_title(f'Class {i }')

# Hide empty subplots if necessary
for i in range(num_classes, num_rows * 5):
    row = i // 5
    col = i % 5
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()
##### PLOT THE MASK THAT IS FED INTO THE MODEL ####


### CHECKING IF DELOITTE GAVE US GOOD MASKS

In [ ]:
orange_im_path = "./data/carseg_data/unused/images/orange_3_doors/with_segmentation"
black_im_path = "./data/carseg_data/unused/images/black_5_doors/with_segmentation"
photo_im_path = "./data/carseg_data/unused/images/photo/with_segmentation"
corrected_mask = "./data/carseg_data/test_arrays_mask_corrected"

mask_name = "photo_"

color_mapping = {
        0: (0,0,0),
        1: (250, 149, 10),
        2: (19, 98, 19),
        3: (249, 249, 10),
        4: (10, 248, 250),
        5: (149, 7, 149),
        6: (5, 249, 9),
        7: (20, 19, 249),
        8: (249, 9, 250),
        9: (0,0,0),
    }

#### CHECKING THE ORANGE CAR DATA 
index = 0
for im in os.listdir(photo_im_path):

    if im.endswith(".jpg") or im.endswith(".png"):
        # Load the image
        
        image_path = os.path.join(photo_im_path, im)
        if "0004" not in image_path:
            if "0028" not in image_path:
                continue
        
        name, ext = im.split('.')
        mask_string = mask_name + name + ".npy"
        mask_path = os.path.join('./data/carseg_data/test_arrays', mask_string)
        correct_mask_path = os.path.join(corrected_mask, mask_string)
        mask = np.load(mask_path)
        mask_split = mask[:, :, 3]
        mask_split = mask_split//10
        mask_split = mask_split.astype(int)
        
        mask_correct = np.load(correct_mask_path)
        mask_split_correct = mask_correct[:, :, 3]
        mask_split_correct = mask_split_correct.astype(int)
        
        img = cv2.imread(image_path)
        resized = resize_with_pad(img, (256, 256))
        img_rgb = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
        
        colorize_segmentation_mask(mask_split, mask=True)
        plt.imshow(img_rgb)
        
        plt.subplot(1, 3, 1)
        plt.imshow(img_rgb)
        plt.title('Intended Mask')

        plt.subplot(1, 3, 2)
        # img_rgb = cv2.cvtColor(model_outputs_images[image_index_to_display], cv2.COLOR_BGR2RGB)
        plt.imshow(colorize_segmentation_mask(mask_split, mask=True))
        plt.title('Numpy Mask')
        
        plt.subplot(1, 3, 3)
        # img_rgb = cv2.cvtColor(model_outputs_images[image_index_to_display], cv2.COLOR_BGR2RGB)
        plt.imshow(colorize_segmentation_mask(mask_split_correct, mask=True))
        plt.title('Corrected Numpy Mask')

        if index == 200:
            break
        index = index + 1
        
    plt.show() 
        